# LPC


In [ ]:
# | default_exp lpc

In [ ]:
# | export

from functools import partial
from typing import Tuple

import jax
import jax.numpy as jnp
import lineax as lx
import numpy as np
import scipy
import scipy.linalg
from jax.scipy.linalg import toeplitz
from jax.typing import ArrayLike

In [ ]:
# | hide
# for testing purposes
jax.config.update("jax_enable_x64", True)

### Utility functions

In [ ]:
# | export


def next_power_of_2(x):
    return 1 if x == 0 else 2 ** (x - 1).bit_length()


def autocorr(
    x: ArrayLike,
    p: int,
    biased: bool = True,
):
    """
    Computes the autocorrelation of a signal x up to lag p.

    Parameters:
    -----------
    x: ArrayLike
        The input signal to compute the autocorrelation of.
    p: int
        The maximum lag to compute the autocorrelation up to.
    """

    X = np.abs(np.fft.rfft(x, next_power_of_2(x.shape[-1] + 1)))
    R = np.fft.irfft(X**2)[..., : p + 1]
    if biased:
        R = R / x.shape[-1]  # Biased autocorrelation
    return R


def unfold(
    x: ArrayLike,  # (batch, time, channels)
    kernel_size: int,
    stride: int = 1,
):
    """
    Unfolds a signal x into a matrix of patches of size kernel_size and stride
    similar to the pytorch unfold function.
    """
    # unfold the y signal
    unfolded = jax.lax.conv_general_dilated_patches(
        x,
        filter_shape=(kernel_size,),
        window_strides=(stride,),
        padding="VALID",
        dimension_numbers=("NTC", "OIT", "NTC"),
    )
    return unfolded


def unfold_np(x, kernel_size, stride=1):
    """
    Unfolds a signal x (batch, time, channels) into a matrix of patches of size kernel_size and stride,
    similar to the pytorch unfold function, using numpy's as_strided function.
    """
    num_patches = (len(x) - kernel_size) // stride + 1

    new_shape = (num_patches, kernel_size)
    new_strides = (x.strides[0] * stride, x.strides[0])

    return np.lib.stride_tricks.as_strided(x, shape=new_shape, strides=new_strides)


def covariance_matrix(
    x: ArrayLike,  # (batch, time)
    p: int,
    L: int = None,
):
    if L is None:
        L = x.shape[-1]

    # We need a (B, T, C) shape, the unfold is done on the T dimension
    # we also need to flip the first dimension to get the autocorrelation
    return jnp.flip(unfold(x[..., None], L - p), axis=1)


def covariance_matrix_np(
    x: np.ndarray,  # (time,)
    p: int,
    L: int = None,
):
    if L is None:
        L = x.shape[-1]

    # We need a (B, T, C) shape, the unfold is done on the T dimension
    # we also need to flip the first dimension to get the autocorrelation
    return np.flip(unfold_np(x, L - p), axis=0)


def discrete_convolution(
    x: np.ndarray,  # (time,)
    a: np.ndarray,  # (time, p)
    zi=None,  # (p,)
):
    """
    Convolves a signal x with a time-varying filter defined by a.
    """

    if zi is None:
        zi = np.zeros(a.shape[-1])
    x_padded = np.concatenate([zi, x])
    e_hat_conv = np.zeros_like(x)

    for n in range(len(x)):
        e_hat_conv[n] = x_padded[n + a.shape[-1]] + np.dot(
            a[n, ::-1], x_padded[n : n + a.shape[-1]]
        )

    return e_hat_conv

In [ ]:
# | test

x = np.arange(10).astype(np.float64)
unfolded_jax = unfold(x[None, :, None], 3)
unfolded_np = unfold_np(x, 3)

X_np = covariance_matrix_np(x, 3, len(x))
X_jax = covariance_matrix(x[None, :], 3, len(x))

assert np.allclose(X_np, X_jax[0])
assert np.allclose(unfolded_np, unfolded_jax[0])

In [ ]:
# | export


@partial(jax.jit, static_argnames=("p",))
def jitted_lstsq(x, p):
    T = x.shape[-1]
    corr = jnp.correlate(x, x, mode="full")[T - 1 : T + p] / T

    a = toeplitz(corr[:-1])
    b = corr[1:]
    sol_lstsq, *_ = jnp.linalg.lstsq(a, b, rcond=None)
    return sol_lstsq


@partial(jax.jit, static_argnames=("p",))
def jitted_linear_solve(x, p):
    T = x.shape[-1]
    corr = jnp.correlate(x, x, mode="full")[T - 1 : T + p] / T

    a = toeplitz(corr[:-1])
    b = corr[1:]

    solver = lx.LU()
    return lx.linear_solve(lx.MatrixLinearOperator(a), b, solver).value


def lpc_cpu_solve(
    x: np.ndarray,  # (time,)
    p: int,  # order
    method="autocorrelation",  # method for the lpc computation
    **kwargs,  # additional arguments for the method
) -> Tuple[np.ndarray, int]:
    """
    Compute the Linear Prediction Coefficients (LPC) of a signal x.
    Based on "Introduction to Digital Speech Processing" by Rabiner and Schafer.

    Parameters:
    ----------
    x: np.ndarray
        The input signal.
    p: int
        The order of the LPC.
    method: str
        The method to compute the LPC. Either "autocorrelation" or "covariance".

    Returns:
    -------
    np.ndarray
        The LPC coefficients.
    int
        The gain (or the square root of the energy) of the residual signal.

    """
    if method == "autocorrelation":
        # biased autocorrelation
        R = autocorr(x, p, **kwargs).astype(
            np.float64
        )  # make sure it's float64 for numerical stability

        # uses levinson-durbin recursion
        a = scipy.linalg.solve_toeplitz(R[:-1], -R[1:])
        g = np.sqrt(R[0] + a @ R[1:])
        return a, g

    elif method == "covariance":
        covar_mat = covariance_matrix_np(x, p, len(x)).astype(
            np.float64
        )  # make sure it's float64 for numerical stability
        R = covar_mat @ covar_mat.T

        # uses cholesky decomposition
        # c, low = cho_factor(R, lower=False)
        # a = cho_solve((c, low), r_0)
        a = scipy.linalg.solve(R[1:, 1:], -R[0, 1:])
        g = np.sqrt(R[0, 0] + a @ R[0, 1:])
        return a, g

In [ ]:
x = np.random.randn(100)

p = 4
a_autocorr, g_autocorr = lpc_cpu_solve(x, p, method="autocorrelation", biased=False)
a_covar, g_covar = lpc_cpu_solve(x, p, method="covariance")

print(a_autocorr, g_autocorr)
print(a_covar, g_covar)

[-0.03891267  0.05897309 -0.11110881 -0.10518435] 8.848879174571616
[-0.05351606  0.04524858 -0.09969181 -0.10755192] 8.666443074154966


In [ ]:
# | export


def linear_prediction_np(
    x: np.ndarray,  # (T,)
    a: np.ndarray,  # (p,)
) -> np.ndarray:
    """
    Computes the linear prediction of a 1D signal. The signal will be padded on the left with zeros.

    Parameters:
    ----------
    x: ArrayLike
        The signal used to compute the linear prediction
    a: ArrayLike
        The coefficients of the linear prediction

    Returns:
    ----------
    ArrayLike:
        The linear prediction of the signal of the same shape as x

    """
    p = a.shape[-1]
    return np.convolve(np.pad(x[:-1], (p, 0)), a, mode="valid")


def linear_prediction(
    x: ArrayLike,  # (T)
    a: ArrayLike,  # (p)
) -> ArrayLike:
    """
    Computes the linear prediction of a 1D signal. The signal will be padded on the left with zeros.

    Parameters:
    ----------
    x: ArrayLike
        The signal used to compute the linear prediction
    a: ArrayLike
        The coefficients of the linear prediction

    Returns:
    ----------
    ArrayLike:
        The linear prediction of the signal of the same shape as x

    """
    p = a.shape[-1]
    return jnp.convolve(
        jnp.pad(x[:-1], (p, 0)),
        a,
        mode="valid",
    )

In [ ]:
# | test

rng = np.random.default_rng(42)
x = rng.normal(0, 1, 100)
a = rng.normal(0, 1, 4)

lp_np = linear_prediction_np(x, a)
lp_jax = linear_prediction(x, a)

assert np.allclose(lp_np, lp_jax)

In [ ]:
# | export


def inverse_filter_np(
    x: np.ndarray,  # (T,)
    a: np.ndarray,  # (p,)
) -> np.ndarray:
    """
    Computes the inverse filter of a signal x using the coefficients a.

    Parameters:
    ----------
    x: ArrayLike
        The signal used to compute the inverse filter
    a: ArrayLike
        The coefficients of the linear prediction

    Returns:
    ----------
    ArrayLike:
        The inverse filter of the signal of the same shape as x
    """

    e_hat = scipy.signal.lfilter(
        b=np.concatenate([[1], a]),
        a=[1],
        x=x,
    )
    return e_hat

In [ ]:
# | export


def coeffs_and_residual(
    y: ArrayLike,  # input signal (time,)
    p: int,  # number of coefficients
    **kwargs,  # additional arguments
) -> Tuple[ArrayLike, ArrayLike, float]:  # coefficients (p+1) and residual and gain
    """
    Utility function to compute the LPC coefficients and the residual.
    """
    if np.all(y == 0):
        a = np.zeros((p + 1))
        a[0] = 1
        e_hat = np.zeros_like(y)
        return a, e_hat, 0

    a, g = lpc_cpu_solve(y, p, **kwargs)
    e_hat = inverse_filter_np(y, a)  # Apply filter to each slice-frame

    return a, e_hat, g

In [ ]:
rng = np.random.default_rng(42)
x = rng.normal(size=1000)
pole = np.array([0.999 * np.exp(1j * np.pi / 4)])
poles = np.concatenate([pole, np.conj(pole)])
b, a = scipy.signal.zpk2tf([], poles, 1)
y = scipy.signal.lfilter(b, a, x)

p = 10
a_autocorr, e_hat_autocorr, _ = coeffs_and_residual(x, p, method="autocorrelation")
a_covar, e_hat_covar, _ = coeffs_and_residual(x, p, method="covariance")